# Run cellpose on a single file

In [1]:
czi_file_path = '/home/ubuntu/Projects/data/uploads/m.zdanowicz@gmail.com/FISH1_BDNF488_1_cLTP_1_CA.czi'
out_mask_filepath = '/home/ubuntu/masks_2D_stitched_FISH1_BDNF488_1_cLTP_3_CA.npy'

In [2]:
# We assume we make use of only one channel (DAPI) from the input
# It is usually the last channel
nuclei_channel = -1 
mask_volume_threshold_of_max = 0.05
diameter = 110

In [3]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
import torch
from cellpose import utils

from pathlib import Path

# use_GPU = utils.use_gpu()
# print('GPU activated? %d'%use_GPU)


In [4]:
assert Path(czi_file_path).suffix == '.czi', czi_file_path
assert Path(czi_file_path).exists(), czi_file_path

In [5]:
base_filename = Path(czi_file_path).stem
base_filename

'FISH1_BDNF488_1_cLTP_1_CA'

## save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [6]:
# TODO(gkk): we might want to save this 
# from cellpose import io

# io.masks_flows_to_seg(imgs, masks, flows, diams, files, channels)

# Read the input data

In [7]:
import tifffile
tifffile.__version__

'2020.12.8'

In [8]:
from tifffile import imread

In [9]:
from aicsimageio import AICSImage

In [10]:
data = AICSImage(czi_file_path).data.squeeze()
print(data.shape)

# Add new axes if the image only has 2 or 3
if len(data.shape) == 2:
    data = data[np.newaxis,np.newaxis,...]
if len(data.shape) == 3:
    data = data[np.newaxis,...]

# swap axes so we have the (Z, channels, Y, X) shape
# TODO: make this automatic
data = np.swapaxes(data, 0, 1)
data.shape

(2, 63, 1024, 1024)


(63, 2, 1024, 1024)

## Cellpose 2D mode
2D planes segmented separately and masks stitched together

In [25]:
z_start_idx, z_end_idx = 0,data.shape[0]
z_start_idx, z_end_idx

(0, 63)

In [27]:
data.shape
imgs = data[z_start_idx:z_end_idx,:,:,:]

In [32]:
imgs = [data[p,nuclei_channel,:,:] for p in range(z_start_idx,z_end_idx)]

In [33]:
len(imgs)

63

In [34]:
channels = len(imgs)*[[0,0]]

In [35]:
imgs[0].shape

(1024, 1024)

In [36]:
# RUN CELLPOSE

from cellpose import models

# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=True, model_type='nuclei')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
# channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(imgs, diameter=110,
                                         cellprob_threshold=0.5, resample=True,
                                         flow_threshold=0.5,
                                         # we disable stitching of masks due to a bug in cellpose
                                         # see below for the workaround
                                         stitch_threshold=0.0,
                                         channels=channels)

TORCH CUDA version not installed/working.
>>>> using CPU
Running test snippet to check if MKL-DNN working
see https://pytorch.org/docs/stable/backends.html?highlight=mkl
** MKL version working - CPU version is sped up. **
processing 63 image(s)
 92%|█████████▏| 58/63 [05:42<00:17,  3.44s/it]

In [ ]:
len(flows)

In [ ]:
flows[0][0].shape

In [ ]:
from cellpose import plot
def display_results(imgs, masks, flows):
    nimg = len(imgs)
    for idx in range(nimg):
        maski = masks[idx]
        flowi = flows[idx]

        fig = plt.figure(figsize=(12,5))
        plot.show_segmentation(fig, imgs[idx], maski, flowi, channels=channels[idx])
        plt.tight_layout()
        plt.show()

In [ ]:
display_results(imgs, 
                masks, 
                # an elaborate way to pick up zeroth element on second dimension of an array
                [flows[idx][0] for idx in range(len(flows))])

## Remap mask ids
Remap mask ids to "compact" integers (from 0..len(unique_ids) range)

In [ ]:
def compact_integer_values(a):
    uniqs = np.unique(a)
    max_value = uniqs.max()
    assert max_value < 100000, max_value # some reasonable threshold that assures we do not allocate unreasonable amounts of memory below
    compact_for_uniqs = np.zeros(uniqs.max()+1, dtype=np.int)
    compact_for_uniqs[uniqs] = range(len(uniqs)) # each uniq receives a new index from the range
    a_compact_values = compact_for_uniqs[a]
    return a_compact_values

In [ ]:
# masks = compact_integer_values(masks)

## Mask volume estimate through voxel counting

In [ ]:
masks_orig = masks
masks = np.copy(masks_orig)

In [ ]:
mask_indices, mask_volume = np.unique(masks,return_counts=True)
mask_indices, mask_volume = mask_indices[1:], mask_volume[1:] # zero is background so we drop it for volume analysis
mask_indices, mask_volume

In [ ]:
plt.bar(range(len(mask_volume)), mask_volume)
plt.title('Volume distribution')
plt.show()

In [ ]:
mask_volume_threshold = mask_volume.max()*mask_volume_threshold_of_max
mask_volume_threshold

In [ ]:
np.sum(mask_volume > mask_volume_threshold)

In [ ]:
masks_indices_above_threshold = mask_indices[mask_volume > mask_volume_threshold]

In [ ]:
masks[np.isin(masks, masks_indices_above_threshold, invert=True)] = 0

In [ ]:
np.unique(masks)

In [ ]:
compact_integer_values(masks)

In [ ]:
masks = masks_orig

Put the exploratory code from the above into a reusable function

In [ ]:
from IPython.display import display
def zero_small_volume_masks(masks, mask_volume_threshold_of_max):
    mask_indices, mask_volume = np.unique(masks,return_counts=True)
    mask_indices, mask_volume = mask_indices[1:], mask_volume[1:] # zero is background so we drop it for volume analysis
    display(mask_indices, mask_volume)
    
    plt.bar(range(len(mask_volume)), mask_volume)
    plt.title('Volume distribution')
    plt.show()
    
    mask_volume_threshold = mask_volume.max()*mask_volume_threshold_of_max
    
    import pandas as pd
    df = pd.DataFrame(mask_volume, columns=['volume'])
    display(df)
    
    df['% of max'] = (df['volume'] / df['volume'].max() * 100).round(1)
    display(df)
    
    masks_indices_above_threshold = mask_indices[mask_volume > mask_volume_threshold]
    masks_large_only = np.copy(masks)
    masks_large_only[np.isin(masks_large_only, masks_indices_above_threshold, invert=True)] = 0
    
    masks_large_only = compact_integer_values(masks_large_only)
    
    display(np.unique(masks_large_only))
    
    return masks_large_only

## Work-around: stitch 2D masks manually
Cellpose has a bug that causes it to blow up if some planes do not have any cells at all (so segmentation masks are empty). In our case, the input file has a few empty planes added as padding during the "alignment" step.

In [ ]:
len(masks)

In [ ]:
from cellpose.utils import stitch3D

In [ ]:
masks = np.array(masks) # with stitching disabled, masks are returned as an array instead of numpy array

### Step 1: find Z-planes with non-empty segmentations

In [ ]:
z_non_zero_idxs = np.any(masks, axis=(1,2))
z_non_zero_idxs

In [ ]:
masks.shape

In [ ]:
np.unique(masks)

In [ ]:
masks[z_non_zero_idxs].shape

### Step 2: stitch non-empty planes

In [ ]:
masks_non_empty_stitched = stitch3D(np.array(masks[z_non_zero_idxs]), stitch_threshold=0.5)

### Step 3: distribute stitched masks back to the original indices of Z-planes

In [ ]:
masks_stitched = np.zeros_like(masks); masks_stitched[z_non_zero_idxs] = masks_non_empty_stitched

In [ ]:
masks_stitched.shape

In [ ]:
masks_stitched = compact_integer_values(masks_stitched)

In [ ]:
np.unique(masks_stitched)

### Remove small volume outliers

In [ ]:
masks_stitched = zero_small_volume_masks(masks_stitched, mask_volume_threshold_of_max)

In [ ]:
np.unique(masks_stitched)

### Save final masks: compacted and without small volume outliers

In [ ]:
np.save(out_mask_filepath, masks_stitched.astype(np.int8))

## run cellpose 3D mode on CZI image

In [ ]:
z_start_idx, z_end_idx = 0,data.shape[0]
z_start_idx, z_end_idx

In [ ]:
imgs = data[z_start_idx:z_end_idx,nuclei_channel,:,:]

In [ ]:
imgs.shape

In [ ]:
channels = len(imgs)*[[0,0]]

In [ ]:
# DISABLED: too expensive to run (for now)
# %%time
# # test 3D stack
# from cellpose import models

# model = models.Cellpose(gpu=True, model_type='cyto')

# # in this example I'm using a random matrix, put your own data here
# # data = np.random.randn(120,512,512).astype(np.float32)
# # data = imread('/home/gkk/ada_lsm_test_squeeze.tif')

# # with 3D you have to set the diameter manually (no auto detect)
# #imgs, diameter=110, #flow_threshddold=, 
# #                                         cellprob_threshold=0.5, resample=True,
# #                                          flow_threshold=0.2,
# #                                         stitch_threshold=0.4,
# #                                         channels=channels
# masks, flows, styles, diams = model.eval(imgs, channels=channels,
#                                          diameter=110, 
#                                          do_3D=True, 
#                                          resample=True,
#                                          cellprob_threshold=0.5,
#                                          flow_threshold=0.5,
# #                                          anisotropy=3.0,
# #                                         boundary_threshold=0.5
# #                                          min_size=80.0,
#                                          batch_size=1)

In [ ]:
# masks.shape

In [ ]:
# np.unique(masks)

In [ ]:
# display_results(imgs, masks, flows[0])

### Remove small volume outliers

In [ ]:
#np.unique(masks)

In [ ]:
#masks = zero_small_volume_masks(masks, mask_volume_threshold_of_max)

In [ ]:
#np.unique(masks)

### Save final masks: compacted and without small volume outliers

In [ ]:
# np.save(f'{basedir_out}/masks_3D_{base_filename}.npy', masks.astype(np.int8))